## For converting JPL GRACE Mascon data from NetCDF (nc) to TIFF (tif) format
__https://grace.jpl.nasa.gov/data/get-data/jpl_global_mascons/__

In [ ]:
import rasterio
from netCDF4 import Dataset
import os
import pandas as pd

def JPL_nc_to_tif(netcdf_file, output_dir, csv_path):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Variable name to extract from the NetCDF file
    variable_name = 'lwe_thickness'

    # Open the NetCDF file for reading
    dataset = Dataset(netcdf_file)

    # Get the variable data
    variable_data = dataset.variables[variable_name]

    # Get the fill value or use a default value
    nodata_value = variable_data._FillValue if hasattr(variable_data, '_FillValue') else -99999.0

    # Get the number of time steps (assuming 'time' is the first dimension)
    dim_size = len(dataset.dimensions['time'])

    # Read the CSV file for naming TIF files
    ls_name = pd.read_csv(csv_path, header=None)

    # Get latitude and longitude information from the dataset
    lon = dataset.variables['lon'][:]
    lat = dataset.variables['lat'][:]
    transform = rasterio.transform.from_origin(lon.min(), lat.max(), lon[1] - lon[0], lat[1] - lat[0])

    # Loop over each time step
    for i in range(dim_size):
        # Extract the slice of data for the current time step
        variable_slice = variable_data[i][::-1, :]

        # Create the TIF file path
        output_tif = os.path.join(output_dir, ls_name.iloc[i, 0] + '.tif')

        # Write the data to a TIF file
        with rasterio.open(output_tif, 'w', driver='GTiff', height=variable_slice.shape[0],
                           width=variable_slice.shape[1], count=1, dtype=variable_slice.dtype,
                           crs='EPSG:4326', transform=transform, nodata=nodata_value) as dst:
            dst.write(variable_slice, 1)

        print(f'TIF file saved at: {output_tif}')

    dataset.close()

netcdf_file = '...your_netcdf_file.nc'
output_dir = '...your_output_directory'
csv_path = '...your_csv_file.csv' ## provided in the folder

JPL_nc_to_tif(netcdf_file, output_dir, csv_path)

## Extracting the scaling factor from JPL mascon GRACE data

In [ ]:
import rasterio
from netCDF4 import Dataset
import os

def JPL_nc_SF_tif(netcdf_file, output_dir, variable_name='scale_factor'):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Open the NetCDF file for reading
    dataset = Dataset(netcdf_file)

    # Get the variable data (scale factor by default)
    variable_data = dataset.variables[variable_name]

    # Get the fill value or use a default value
    nodata_value = variable_data._FillValue if hasattr(variable_data, '_FillValue') else -99999.0

    # Reverse the data for correct orientation (latitude usually needs to be flipped)
    variable_slice = variable_data[::-1, :]

    # Get the metadata (coordinates, spatial reference system, etc.) from the NetCDF file
    lon = dataset.variables['lon'][:]
    lat = dataset.variables['lat'][:]
    transform = rasterio.transform.from_origin(lon.min(), lat.max(), lon[1] - lon[0], lat[1] - lat[0])

    # Create the TIF file path
    output_tif = os.path.join(output_dir, f'{variable_name}.tif')

    # Write the variable data to a TIF file
    with rasterio.open(output_tif, 'w', driver='GTiff', height=variable_slice.shape[0],
                            width=variable_slice.shape[1], count=1, dtype=variable_slice.dtype,
                            crs='EPSG:4326', transform=transform, nodata=nodata_value) as dst:
        dst.write(variable_slice, 1)

    print(f'TIF file saved at: {output_tif}')

    # Close the NetCDF dataset
    dataset.close()

netcdf_file = '....your_netcdf_file.nc'
output_dir = '...your_output_directory'
JPL_nc_SF_tif(netcdf_file, output_dir)


## JPL GRACE TWSA = mascon * scale_factor

In [ ]:
import rasterio
import os

def JPL_TWSA_SF(input_dir, scale_factor_file, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Get the list of TIF files in the input directory
    list_files = [f for f in os.listdir(input_dir) if f.endswith('.tif')]

    # Open and read the scale factor file
    with rasterio.open(scale_factor_file) as sf:
        sf_data = sf.read(1, masked=True)

    # Loop through each file in the list
    for fl in list_files:
        input_tiff_path = os.path.join(input_dir, fl)

        # Open the input TIF file
        with rasterio.open(input_tiff_path, 'r') as src:
            # Read the raster data
            raster_data = src.read(1, masked=True)

            # Multiply the raster data with the scale factor
            result = raster_data * sf_data

            # Copy and update metadata for the output TIF file
            meta = src.meta.copy()
            meta.update(nodata=src.nodata)

        # Define output TIF file path
        output_tiff_path = os.path.join(output_dir, fl)

        # Write the scaled data to the output TIF file
        with rasterio.open(output_tiff_path, 'w', **meta) as dst:
            dst.write(result, 1)

        # Print the path of the saved file
        print(f'Saved file: {output_tiff_path}')

input_directory = '...path_to_input_tifs'
scale_factor_path = '...path_to_scale_factor_tif'
output_directory = '...path_to_output_directory'

JPL_TWSA_SF(input_directory, scale_factor_path, output_directory)


## For converting CSR GRACE Mascon data from NetCDF (nc) to TIFF (tif) format
__https://grace.jpl.nasa.gov/data/get-data/__

In [ ]:
import netCDF4 as nc
import rasterio
import pandas as pd
import os
from netCDF4 import num2date

def CSR_nc_tif(netcdf_file, output_dir, csv_file):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Variable name to extract from the NetCDF file
    variable_name = 'lwe_thickness'

    # Open the NetCDF file for reading
    dataset = nc.Dataset(netcdf_file)

    # Get the variable data
    variable_data = dataset.variables[variable_name]
    
    # Get the nodata value
    nodata_value = -99999.0

    # Get the dimension size (assuming it represents time)
    dim_size = len(dataset.dimensions['time'])

    # Reading CSV file containing names for each timestep
    ls_name = pd.read_csv(csv_file, header=None)

    # Loop over each index of the time dimension
    for i in range(dim_size):
        # Read the variable data for the current time slice and reverse latitudes
        variable_slice = variable_data[i][::-1, :]

        # Get the longitude and latitude coordinates from the NetCDF file
        lon = dataset.variables['lon'][:]
        lat = dataset.variables['lat'][:]

        # Define the transform based on the spatial information
        transform = rasterio.transform.from_origin(lon.min(), lat.max(), lon[1] - lon[0], lat[1] - lat[0])

        # Create the TIF file path using the corresponding name from the CSV file
        output_tif = os.path.join(output_dir, f"{ls_name.iloc[i, 0]}.tif")

        # Create the TIF file and write the variable data
        with rasterio.open(output_tif, 'w', driver='GTiff', height=variable_slice.shape[0],
                           width=variable_slice.shape[1], count=1, dtype=variable_slice.dtype,
                           crs='EPSG:4326', transform=transform, nodata=nodata_value) as dst:
            dst.write(variable_slice, 1)

        print(f'TIF file saved at: {output_tif}')

    # Close the NetCDF dataset
    dataset.close()

netcdf_file_path = '...path_to_netCDF_file.nc'
output_directory = '....path_to_output_directory'
csv_file_path = '....path_to_csv_file.csv'

CSR_nc_tif(netcdf_file_path, output_directory, csv_file_path)


## For converting GSFC GRACE Mascon data from NetCDF (nc) to TIFF (tif) format
__https://earth.gsfc.nasa.gov/geo/data/grace-mascons__

In [ ]:
import netCDF4 as nc
import rasterio
import pandas as pd
import os

def GSFC_nc_tif(netcdf_file, output_dir, csv_file):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Variable name to extract from the NetCDF file
    variable_name = 'lwe_thickness'

    # Open the NetCDF file for reading
    dataset = nc.Dataset(netcdf_file)

    # Get the variable data
    variable_data = dataset.variables[variable_name]

    # Set nodata value
    nodata_value = -99999.0

    # Get the dimension size (assuming it represents time)
    dim_size = len(dataset.dimensions['time'])

    # Read the CSV file containing names for each time step
    ls_name = pd.read_csv(csv_file, header=None)

    # Loop over each index of the time dimension
    for i in range(dim_size):
        # Read the variable data for the current time slice and reverse latitudes
        variable_slice = variable_data[i][::-1, :]

        # Get longitude and latitude coordinates from the NetCDF file
        lon = dataset.variables['lon'][:]
        lat = dataset.variables['lat'][:]

        # Define the transform based on the spatial information
        transform = rasterio.transform.from_origin(lon.min(), lat.max(), lon[1] - lon[0], lat[1] - lat[0])

        # Create the TIF file path using the corresponding name from the CSV file
        output_tif = os.path.join(output_dir, f"{ls_name.iloc[i, 0]}.tif")

        # Create the TIF file and write the variable data
        with rasterio.open(output_tif, 'w', driver='GTiff', height=variable_slice.shape[0],
                           width=variable_slice.shape[1], count=1, dtype=variable_slice.dtype,
                           crs='EPSG:4326', transform=transform, nodata=nodata_value) as dst:
            dst.write(variable_slice, 1)

        print(f'TIF file saved at: {output_tif}')

    # Close the NetCDF dataset
    dataset.close()

netcdf_file = '...path_to_netCDF_file.nc'
output_dir = '...path_to_output_directory'
csv_file = '...path_to_csv_file.csv'

GSFC_nc_tif(netcdf_file, output_dir, csv_file)


## Resampling the tif file

In [ ]:
import rasterio
from rasterio.enums import Resampling
import os

def resample_tif(input_tif, output_tif, new_resolution):
    # Open the original TIFF file
    with rasterio.open(input_tif) as src:
        # Calculate the new dimensions
        width = int(src.width * (src.res[0] / new_resolution))
        height = int(src.height * (src.res[1] / new_resolution))

        # Define the transform for the new resolution
        transform = src.transform * src.transform.scale(
            (src.width / width),
            (src.height / height)
        )

        # Read the data from the original TIFF file
        data = src.read(
            out_shape=(src.count, height, width),
            resampling=Resampling.bilinear  # You can change to nearest, cubic, etc.
        )

        # Update metadata for the output file
        meta = src.meta.copy()
        meta.update({
            'driver': 'GTiff',
            'height': height,
            'width': width,
            'transform': transform,
            'crs': src.crs  # Keep the same coordinate reference system
        })

        # Write the resampled data to a new TIFF file
        with rasterio.open(output_tif, 'w', **meta) as dst:
            dst.write(data)

    print(f'Resampled TIFF saved at: {output_tif}')

# Example usage
path_folder = r'....\GRACE data\JPL'
output_folder = os.path.join(path_folder, "Resample")
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
new_resolution = 0.25  # Target resolution in degree

# Get a list of TIFF files in the input folder
tiff_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]
# Iterate over all TIFF files in the input folder
for filename in tiff_files:
    input_tif = os.path.join(path_folder, filename)
    output_tif = os.path.join(output_folder, filename)
    resample_tif(input_tif, output_tif, new_resolution)

## Average of three mascon solutions

In [ ]:
import rasterio
import os
import numpy as np

def average_mascon_solutions(JPL_path, CSR_path, GSFC_path, dst_path):
    """
    Function to compute the average of mascon solutions (GeoTIFFs) from JPL, CSR, and GSFC sources.
    
    Parameters:
    - JPL_path: Path to JPL GeoTIFF files.
    - CSR_path: Path to CSR GeoTIFF files.
    - GSFC_path: Path to GSFC GeoTIFF files.
    - dst_path: Output directory where the averaged GeoTIFFs will be saved.
    """
    
    # List of paths for input directories
    file_paths = [JPL_path, CSR_path, GSFC_path]
    
    # Get the list of GeoTIFF files from JPL (assuming all folders have the same file names)
    list_files = [f for f in os.listdir(JPL_path) if f.endswith('.tif')]
    list_files.sort()  # Sort the files to ensure proper matching

    # Create output directory if it doesn't exist
    if not os.path.exists(dst_path):
        os.makedirs(dst_path)
    
    # Read metadata from the first file to set up the output
    with rasterio.open(os.path.join(JPL_path, list_files[0])) as src:
        profile = src.profile  # Metadata profile (CRS, dtype, etc.)
        dtype = src.dtypes[0]  # Data type
        nodata = src.nodatavals[0]  # NoData value
        shape = src.shape  # Shape of the raster
    
    # Loop over each GeoTIFF file
    for fl in list_files:
        raster_files = [os.path.join(path, fl) for path in file_paths if os.path.exists(os.path.join(path, fl))]
        
        if len(raster_files) < 3:
            print(f"Skipping {fl} due to missing files in one or more directories.")
            continue
        
        # Initialize a list to store raster data
        image_data = []
        
        # Read and append data from all three rasters
        for file in raster_files:
            with rasterio.open(file) as src:
                data = src.read(1, masked=True)  # Read band 1 with masking
                image_data.append(data)
        
        # Convert the list of image data to a NumPy masked array and compute the average
        image_data = np.ma.array(image_data)
        average_image = np.ma.mean(image_data, axis=0)
        
        # Update metadata profile for the output raster
        profile.update(dtype=dtype, nodata=nodata)
        
        # Define the output file path
        output_file = os.path.join(dst_path, fl)
        
        # Write the averaged data to a new GeoTIFF file
        with rasterio.open(output_file, 'w', **profile) as dst:
            dst.write(average_image, 1)
        
        print(f"Saved averaged GeoTIFF: {output_file}")

# Define paths
JPL_path = r"...\GRACE data\JPL\\"
CSR_path = r"...\GRACE data\CSR\\"
GSFC_path = r"...\GRACE data\GSFC\\"
dst_path = r"...\GRACE data\Average\\"

# Run the function
average_mascon_solutions(JPL_path, CSR_path, GSFC_path, dst_path)


## Create raster layer normalized Day of Year (nDoY)

In [ ]:
import os
import rasterio
import pandas as pd
import numpy as np
from datetime import datetime
def normalized_day_of_year(year, month, day):

    # Create a datetime object for the input date
    date_object = datetime(year, month, day)

    # Calculate the day of the year (DOY) as an integer value (ranging from 1 to 365/366)
    day_of_year = date_object.timetuple().tm_yday

    # Calculate the total number of days in the year
    total_days_in_year = 365 if not is_leap_year(year) else 366

    # Calculate the normalized day of the year as a fraction (ranging from 0 to 1)
    normalized_doy = (day_of_year - 1) / total_days_in_year

    return normalized_doy

def is_leap_year(year):
    # Function to check if a year is a leap year
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def array_ndoy(start_date, end_date, row, col):
    # Create the date range with a monthly difference starting from the 15th day of each month
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    # Convert the date range to a list
    date_range_list = [str(date).strip('00:00:00').replace(" ", "") for date in date_range.to_list()]

    # Creat the array of ndoy
    ndoy = np.zeros((len(date_range_list), row, col))
    for index, date in enumerate(date_range_list):
        for r in range(row):
            for c in range(col):
                # Convert the string date to a datetime object
                date_obj = datetime.strptime(str(date), "%Y-%m-%d")
                # Extract the components
                day = int(date_obj.day/2)
                month = date_obj.month
                year = date_obj.year

                normalized_doy = normalized_day_of_year(year, month, day)
                ndoy[index, r, c] = normalized_doy
    return ndoy

################################################################################################################################

ndoy_path = r'...\GRACE data\NDOY\\'
if not os.path.exists(ndoy_path):
    os.makedirs(ndoy_path)

ref_path = r'...\Model data\ET\\'
ref_files = [f for f in os.listdir(os.path.join(ref_path)) if f.endswith('.tif')]
ref_files.sort()

with rasterio.open(os.path.join(ref_path, ref_files[0])) as ref:
    input_data = ref.read(1, masked = True)
    profile = ref.profile
    
row, col = input_data.shape
# Define the start and end dates
start_date = "2002-04-01"
end_date = "2023-05-31"
    
ndoy = array_ndoy(start_date, end_date, row, col)    
n_sample = ndoy.shape[0]    
for sample in range(n_sample):
    with rasterio.open(ndoy_path+ref_files[sample], "w", **profile) as dst:
        dst.write(np.array(ndoy[sample]), 1)
        print(ref_files[sample])



## Decompose all input variables into their residual and slope components

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import re
import rasterio
import numpy as np
from statsmodels.tsa.seasonal import STL
import pymannkendall as mk

def create_month_range(start_date, end_date):
    """Generates a list of monthly dates between start and end dates in 'YYYY-MM' format."""
    monthly_dates = []
    current_date = start_date
    while current_date <= end_date:
        monthly_dates.append(current_date.strftime('%Y-%m'))
        current_date += relativedelta(months=1)
    return monthly_dates

def atoi(text):
    """Helper function for sorting numbers in text."""
    return int(text) if text.isdigit() else text

def natural_keys(text):
    """Natural sorting key."""
    return [atoi(c) for c in re.split(r'(\d+)', text)]

def data_retrieve_withna(fol_files, date_range, array):
    """Retrieves data, introducing NaN values for missing files in the date range."""
    data_with_missing = []
    i = 0
    for date in date_range:
        if date + ".tif" in fol_files:
            data_with_missing.append(array[i])
            i += 1
        else:
            data_with_missing.append(np.nan)
    return data_with_missing

def stl_method(data_without_na, array_miss):
    """Performs STL decomposition and returns the linear trend and residuals."""
    stl_result = STL(data_without_na, period=12, seasonal=13, trend_deg=0, seasonal_deg=0, low_pass_deg=0, robust=True).fit()
    trend_component = stl_result.trend
    mk_result = mk.original_test(trend_component)
    
    linear_trend = [(x + 1) * mk_result.slope + mk_result.intercept for x in range(len(array_miss))]
    residual = np.array(array_miss) - np.array(linear_trend)
    
    return linear_trend, residual

def get_folder_list(path):
    """Returns a list of subfolders in the specified path."""
    return [entry.name for entry in os.scandir(path) if entry.is_dir()]

def decompose_all_data(path):
    folder_list = get_folder_list(path)
    
    for folder in folder_list:
        fol_path = os.path.join(path, folder)
        fol_files = sorted([f for f in os.listdir(fol_path) if f.endswith('.tif')], key=natural_keys)

        # Read metadata from the first file
        with rasterio.open(os.path.join(fol_path, fol_files[0])) as ref:
            ref_data = ref.read(1, masked=True)
            profile = ref.profile
            nodata = ref.nodata

        # Date range
        start_date = datetime(2002, 4, 1)
        end_date = datetime(2023, 5, 31)
        date_range = create_month_range(start_date, end_date)

        # Read all fol files into an array
        raster_array = [rasterio.open(os.path.join(fol_path, filename)).read(1, masked=True) for filename in fol_files]
        raster_array = np.array(raster_array)

        n_sample, row, col = raster_array.shape

        slope_path = os.path.join(fol_path, "Slope")
        os.makedirs(slope_path, exist_ok=True)

        residual_path = os.path.join(gfol_path, "Residual")
        os.makedirs(residual_path, exist_ok=True)

        lt, rd = [], []

        for r in range(row):
            for c in range(col):
                pixel_data = raster_array[:, r, c]
                if pixel_data[0] != nodata:
                    array_miss = data_retrieve_withna(fol_files, date_range, pixel_data)
                    data_without_na = [x for x in array_miss if not np.isnan(x)]
                    linear_trend, residual = stl_method(data_without_na, array_miss)
                    lt.append(linear_trend)
                    rd.append(residual)
                else:
                    lt.append([nodata] * len(date_range))
                    rd.append([nodata] * len(date_range))

        lt = np.array(lt).reshape(row, col, len(date_range))
        rd = np.array(rd).reshape(row, col, len(date_range))

        # Write output files
        for i, date in enumerate(date_range):
            slope_file = os.path.join(slope_path, f"{date}.tif")
            residual_file = os.path.join(residual_path, f"{date}.tif")
            
            with rasterio.open(slope_file, 'w', **profile) as dst:
                dst.write(lt[:, :, i], 1)
            with rasterio.open(residual_file, 'w', **profile) as dst:
                dst.write(rd[:, :, i], 1)
            
            print(f"Processed {date} successfully.")

# Define path 
path = r"...\Model without slope\Data\\"
decompose_all_data(path)


# For the decomposition of GRACE data into slope and residual components

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import re
import rasterio
import numpy as np
from statsmodels.tsa.seasonal import STL
import pymannkendall as mk

# Function to create a monthly date range between start and end dates
def create_month_range(start_date, end_date):
    current_date = start_date
    monthly_dates = []
    while current_date <= end_date:
        monthly_dates.append(current_date.strftime('%Y-%m'))
        current_date += relativedelta(months=1)
    return monthly_dates

# Sorting function for filenames
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

# Function to retrieve data, filling missing values with NaN
def data_retrieve_withna(GRACE_files, date_range, array):
    data_with_missing = []
    i = 0
    for file in date_range:
        if file in GRACE_files:
            data_with_missing.append(array[i])
            i += 1
        else:
            data_with_missing.append(np.nan)
    return data_with_missing

# Function to apply the STL decomposition and calculate trends
def stl_method(data_without_na, array_miss):
    decomposition = STL(data_without_na, period=12, seasonal=13, trend_deg=0, seasonal_deg=0, low_pass_deg=0, robust=True).fit()
    trend_component = decomposition.trend

    # Perform Mann-Kendall trend test
    mk_result = mk.original_test(trend_component)
    linear_trend = [(x + 1) * mk_result.slope + mk_result.intercept for x in range(len(array_miss))]

    # Calculate residuals
    residuals = np.array(array_miss) - np.array(linear_trend)
    
    return linear_trend, residuals

def decompose_grace_data(GRACE_path):
    # read GRACE files
    GRACE_files = [f for f in os.listdir(GRACE_path) if f.endswith('.tif')]
    GRACE_files.sort(key=natural_keys)

    # Read reference data and set metadata profile
    with rasterio.open(os.path.join(GRACE_path, GRACE_files[0])) as ref:
        ref_data = ref.read(1, masked=True)
        profile = ref.profile
        nodata = ref.nodata

    # Define start and end dates
    start_date = datetime(2002, 4, 1)
    end_date = datetime(2023, 5, 31)

    # Create date range for the files
    date_range = create_month_range(start_date, end_date)
    date_range_files = [f + ".tif" for f in date_range]

    # Read all GRACE raster data into a list
    raster_array = []
    for filename in GRACE_files:
        with rasterio.open(os.path.join(GRACE_path, filename)) as src:
            raster_array.append(src.read(1, masked=True))

    # Reshape the raster array
    n_sample, row, col = np.array(raster_array).shape

    # Create directories for output if they don't exist
    slope_path = os.path.join(GRACE_path, "Slope")
    residual_path = os.path.join(GRACE_path, "Residual")
    os.makedirs(slope_path, exist_ok=True)
    os.makedirs(residual_path, exist_ok=True)

    # Initialize lists for linear trends and residuals
    lt = []
    rd = []

    # Process each pixel
    for r in range(row):
        for c in range(col):
            if np.array(raster_array)[0, r, c] != nodata:
                array_miss = data_retrieve_withna(GRACE_files, date_range_files, np.array(raster_array)[:, r, c])
                data_without_na = [x for x in array_miss if not np.isnan(x)]
                linear_trend, residual = stl_method(data_without_na, array_miss)
            else:
                array_miss = [nodata for _ in range(len(date_range_files))]
                linear_trend, residual = np.array(array_miss), np.array(array_miss)

            lt.append(linear_trend)
            rd.append(residual)

    # Reshape lists to arrays
    lt = np.array(lt).reshape(row, col, len(date_range_files))
    rd = np.array(rd).reshape(row, col, len(date_range_files))

    # Save linear trend and residual rasters
    for i in range(len(date_range_files)):
        with rasterio.open(os.path.join(slope_path, date_range_files[i]), "w", **profile) as dst:
            dst.write(lt[:, :, i], 1)
        if date_range_files[i] in GRACE_files:
            with rasterio.open(os.path.join(residual_path, date_range_files[i]), "w", **profile) as dst:
                dst.write(rd[:, :, i], 1)
        print(f"{i} - {date_range_files[i]} saved successfully")

        # Define paths 
path = r"...\GRACE TWSA\\"
decompose_grace_data(path)

## Apply the MissForest model for filling spatial data gaps

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from osgeo import gdal
import re
import os
import rasterio
import numpy as np
from missingpy import KNNImputer, MissForest
from scipy.stats import pearsonr
from matplotlib.ticker import MultipleLocator

# Helper function to convert strings to integers if applicable
def atoi(text):
    return int(text) if text.isdigit() else text

# Function to create natural sorting keys
def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

# Function to retrieve a list of folders from a given path
def get_folder_list(path):
    folder_list = [entry.name for entry in os.scandir(path) if entry.is_dir()]
    return folder_list

# Main path for data
path = r'...\Model without slope\Data\\'

# Get the list of variable folders and remove the target variable folder ('GRACE TWSA')
vars_list = get_folder_list(path)
target_var = 'GRACE TWSA'
vars_list.remove(target_var)
input_vars = vars_list
print(input_vars)

# Get and sort target files
target_files = [f for f in os.listdir(os.path.join(path, target_var)) if f.endswith('.tif')]
target_files.sort(key=natural_keys)


In [ ]:
new_target_files

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import re
import os
import rasterio
import numpy as np
from missingpy import MissForest
from scipy.stats import pearsonr
from matplotlib.ticker import MultipleLocator

# Helper function to convert strings to integers if applicable
def atoi(text):
    return int(text) if text.isdigit() else text

# Function to create natural sorting keys for filenames
def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

# Function to retrieve a list of folders from a given path
def get_folder_list(path):
    return [entry.name for entry in os.scandir(path) if entry.is_dir()]

# Main path for data
path = r'...\Model without slope\Data\\'

# Get the list of variable folders and remove the target variable folder ('GRACE TWSA')
vars_list = get_folder_list(path)
target_var = 'GRACE TWSA'
vars_list.remove(target_var)  # Remove the target variable from input vars
input_vars = vars_list
print(f"Input variables: {input_vars}")

# Get and sort target files
target_files = [f for f in os.listdir(os.path.join(path, target_var)) if f.endswith('.tif')]
target_files.sort(key=natural_keys)

# Get and sort input variable 1 files (assuming the first folder in input_vars is needed)
var1_files = [f for f in os.listdir(os.path.join(path, input_vars[0])) if f.endswith('.tif')]
var1_files.sort(key=natural_keys)

# Initialize the dictionary to store data arrays
data_array = {}

# Reference data for initializing raster shape and profile
with rasterio.open(os.path.join(path, target_var, target_files[0])) as ref:
    ref_data = ref.read(1, masked=True)  # Read masked raster data (NaN where no data)
    profile = ref.profile  # Save the profile for later use (if needed)

# Loop through the input variable 1 files and process each file
for index, filename in enumerate(var1_files):
    raster_array = []

    # Check if the file exists in the target files
    if filename in target_files:
        for var in input_vars:
            # Read input images from respective folders using rasterio
            with rasterio.open(os.path.join(path, var, filename)) as src:
                input_data = src.read(1, masked=True)  # Read data as masked array
                raster_array.append(input_data)
        
        # Read the target data
        with rasterio.open(os.path.join(path, target_var, filename)) as src_t:
            target_data = src_t.read(1, masked=True)
            raster_array.append(target_data)
        
        # Stack the input and target data along the third dimension
        data_array[index] = np.stack(raster_array, axis=0)
    else:
        for var in input_vars:
            # Read input data for missing target file
            with rasterio.open(os.path.join(path, var, filename)) as src:
                input_data = src.read(1, masked=True)
                raster_array.append(input_data)
        
        # Create an array of NaNs for missing target data
        target_data = np.full(ref_data.shape, np.nan)
        raster_array.append(target_data)
        
        # Stack the input data along with NaN target data
        data_array[index] = np.stack(raster_array, axis=0)

# Optional: Print the keys of data_array to confirm files processed
print(f"Processed data indices: {list(data_array.keys())}")


## Data standarization

In [ ]:
# Extract the keys from the data array and get shape parameters
data_samples = list(data_array.keys())
n_features, row, col = data_array[0].shape

print('n_feature:', n_features, 'Row:', row, 'Col:', col)

# Initialize dictionaries and lists for normalized data, mean, and standard deviation
data_norm = {}
data_mean_array = []
data_std_array = []

# Normalize data for each feature
for feature in range(n_features):
    for r in range(row):
        for c in range(col):
            data_temp = [data_array[sample][feature, r, c] for sample in data_samples]
            
            if feature != n_features - 1:
                # Normalize data only if not the last feature
                if np.array(data_temp)[0] != 9999.0:
                    data_mean = np.mean(data_temp)
                    data_std = np.std(data_temp)
                    if data_std != 0.0:
                        data_normalized = (data_temp - data_mean) / data_std
                    else:
                        data_normalized = data_temp - data_mean
                    data_norm[(feature, r, c)] = data_normalized
                else:
                    data_norm[(feature, r, c)] = np.array(data_temp)
            else:
                # Special case for the last feature
                if np.array(data_temp)[0] != -99999.0:
                    data_mean = np.nanmean(data_temp)
                    data_std = np.nanstd(data_temp)
                    data_mean_array.append(data_mean)
                    if data_std != 0.0:
                        data_normalized = (data_temp - data_mean) / data_std
                    else:
                        data_normalized = data_temp - data_mean
                    data_std_array.append(data_std)
                    data_norm[(feature, r, c)] = data_normalized
                else:
                    # Handle the case where the first element is -99999.0
                    data_mean_array.append(np.array(data_temp)[0])
                    data_std_array.append(np.array(data_temp)[0])
                    data_norm[(feature, r, c)] = np.array(data_temp)

# Prepare normalized images and mean/std arrays for further processing
data_norm_image = np.array([data_norm[k] for k in data_norm.keys()]).reshape(n_features, row, col, -1)
data_mean_array = np.array(data_mean_array).reshape(row, col)
data_std_array = np.array(data_std_array).reshape(row, col)

# Optional: Print shapes of normalized images, means, and stds for confirmation
print('Normalized image shape:', data_norm_image.shape)
print('Mean array shape:', data_mean_array.shape)
print('Std array shape:', data_std_array.shape)


## MissForest with standarization

In [ ]:
import os
import numpy as np
import rasterio
from missingpy import MissForest

# Define the output directory for imputed data
imputed_path = r'...\Model without slope\MissForest\\' 

# Create the output directory if it doesn't exist
os.makedirs(imputed_path, exist_ok=True)

# Initialize the MissForest imputer
imputer = MissForest()
n_features, row, col, n_sample = data_norm_image.shape
imputed = []

# Iterate through each pixel (row, col) to perform imputation
for r in range(row):
    for c in range(col):
        data = data_norm_image[:, r, c, :].T  # Transpose for easier processing

        # Check if the first element is valid
        if data[0, 0] != 9999.0:
            # Perform imputation
            temp = imputer.fit_transform(data)  
            # Scale the imputed last layer back to original units
            imputed_value = (temp[:, -1] * data_std_array[r, c] + data_mean_array[r, c])
        else:
            # Handle missing data by replacing NaNs with the mean value
            temp_na = data[:, -1]
            temp_na[np.isnan(temp_na)] = np.nanmean(temp_na)  # Replace NaNs with the mean value
            imputed_value = temp_na

        imputed.append(imputed_value)

# Convert the list of imputed values to a NumPy array
imputed_array = np.array(imputed)  # Shape: [row*col, n_sample]

# Update the profile for output raster files
profile.update(dtype='float64')

# Write each imputed sample to a raster file
for sample in range(n_sample):
    temp = imputed_array[:, sample].reshape(row, col)
    output_file_path = os.path.join(imputed_path, var1_files[sample])
    with rasterio.open(output_file_path, "w", **profile) as dst:
        dst.write(temp, 1)
        print(f'Written file: {var1_files[sample]}')


## Added slope to the predicted value

In [ ]:
import rasterio
import numpy as np
import os

# Define paths for slope and filled data
path_slope = r"...\Model without slope\GRACE TWSA\Slope\\"
path_filled_missforest = r"...\MissForest\\"

# List of input GeoTIFF files
list_files = sorted([f for f in os.listdir(path_slope) if f.endswith('.tif')], key=natural_keys)

# Destination path for output files
path_filled_with_slope = os.path.join(path_filled_missforest, "Filled")

# Create the destination directory if it doesn't exist
os.makedirs(path_filled_with_slope, exist_ok=True)

# Process each file
for f in list_files:
    # Construct file paths
    slope_file = os.path.join(path_slope, f)
    filled_file = os.path.join(path_filled_missforest, f)

    # Read slope data
    with rasterio.open(slope_file) as sl:
        slp = sl.read(1)

    # Read filled data
    with rasterio.open(filled_file) as fl:
        flp = fl.read(1)

    # Create a mask for NoData values
    nodata_mask = (flp == fl.nodata)

    # Add the slope to the filled data, excluding NoData values
    result = np.where(nodata_mask, flp, slp + flp)

    # Update metadata and specify NoData value
    meta = fl.meta.copy()
    meta.update(nodata=fl.nodata)

    # Create a new GeoTIFF file to store the result
    output_file = os.path.join(path_filled_with_slope, f)
    with rasterio.open(output_file, 'w', **meta) as dst:
        dst.write(result.astype(meta['dtype']), 1)

    print(f'Written output file: {output_file}')


## Plot filled with imputed values and actual GRACE TWSA

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

def plot_figures(results_data):
    plt_row = len(results_data.keys())
    # Set up the figure size
    figure_size = (12, 4)  # Width, Height
    fig, ax = plt.subplots(figsize=figure_size)
    fig.tight_layout(pad=4)  # Set the padding in inches

    # Extract the data for plotting
    y = np.array(results_data['MissForest'][0])
    x = np.arange(1, len(y) + 1)  # Generate x values
    valid_indices = np.where(np.isnan(y))[0] + 1  # Identify NaN values for valid indices

    # Plot predicted and observed values
    ax.plot(x, np.array(results_data['MissForest'][1]), color='r', linestyle='-', marker='*', linewidth=1.5, label='Predicted')
    ax.plot(x, y, color='b', marker='o', linewidth=1.5, linestyle='-', label='Observed')
    
    # Add bars to indicate gaps in data
    ax.bar(valid_indices, y[valid_indices - 1] - 0.1 * max(y), width=0.8, color='maroon', alpha=0.3, label='Gap in Data')
    ax.bar(valid_indices, y[valid_indices - 1] + 0.1 * max(y), width=0.8, color='maroon', alpha=0.3)

    # Configure axes
    ax.set_xlim(-1, len(y) + 1)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3)
    ax.xaxis.set_major_locator(MultipleLocator(10))
    ax.set_ylabel('EWH [cm]', fontsize=12)
    ax.set_xlabel('Time (months)', fontsize=12)
    ax.grid(True)

    # Save the graph as TIFF with 300 DPI
    save_path = r'...\Indus model\\'
    os.makedirs(save_path, exist_ok=True)
    fig.savefig(os.path.join(save_path, 'MissForest_complete.tif'), dpi=300, format='tif', bbox_inches='tight')
    
    plt.show()               

# Set paths for actual and filled data
path_actual = r"...\Model without slope\GRACE TWSA\\"
path_filled = path_filled_with_slope  # Assuming `path_filled_with_slope` is defined earlier

# List actual files
actual_files = sorted([f for f in os.listdir(path_actual) if f.endswith('.tif')], key=natural_keys)

# List filled files
list_files = sorted([f for f in os.listdir(path_slope) if f.endswith('.tif')], key=natural_keys)

mean_actual = []
mean_filled = []
results_com = {}

# Calculate means for actual and filled data
for file in list_files:
    if file in actual_files:
        mean_actual.append(calculate_mean_value(os.path.join(path_actual, file)))
    else:
        mean_actual.append(np.nan)
        
    mean_filled.append(calculate_mean_value(os.path.join(path_filled, file)))

# Stack the results
results_com[fol] = np.stack((mean_actual, mean_filled), axis=0)

# Plot the figures
plot_figures(results_com)
